In [ ]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

Let's import some packages first.

In [ ]:
#| output: false
using GeoMakie
using GeoInterfaceMakie
using GeoInterface
using CairoMakie
using Shapefile
using DataFrames
using DataFramesMeta
using StringEncodings
using Pkg.Artifacts

If you want to run this interactively, you can replace `CairoMakie` with `GLMakie`, i.e.

```diff
- import CairoMakie
+ import GLMakie
```

## Data

Representations of Canada's national road network are available from [Statistics Canada](https://www12.statcan.gc.ca/census-recensement/2011/geo/RNF-FRR/index-eng.cfm).

In [ ]:
artifact_roadnetwork = Pkg.Artifacts.ensure_artifact_installed("roadnetwork", joinpath(@__DIR__, "Artifacts.toml"))
path = joinpath(artifact_roadnetwork, "lrnf000r21a_e.shp")
@time gdf = DataFrame(Shapefile.Table(path));
@show size(gdf)
first(gdf, 1)

The documentation says `CSDNAME` is the "Census subdivision name", which seems to map to cities.

Let's convert it to a proper encoding first:

In [ ]:
latin1_to_utf8(s) = decode(Vector{UInt8}(String(coalesce(s, ""))), "Windows-1252")
@time @rtransform! gdf begin
  :CSDNAME_L_UTF8 = latin1_to_utf8(:CSDNAME_L)
  :CSDNAME_R_UTF8 = latin1_to_utf8(:CSDNAME_R)
end
nothing #| hide_line

## Visualizations

We can now create a plot for each city using Makie:

In [ ]:
#| code-fold: true
function plot_city(gdf, city_name)
  df = @rsubset gdf (:CSDNAME_L_UTF8 == city_name || :CSDNAME_R_UTF8 == city_name)
  empty_theme = Theme(
    fonts=(; weird="Blackchancery"),
    fontsize=32,
    Axis=(
      backgroundcolor=:transparent,
      leftspinevisible=false,
      rightspinevisible=false,
      bottomspinevisible=false,
      topspinevisible=false,
      xticklabelsvisible=false,
      yticklabelsvisible=false,
      xgridcolor=:transparent,
      ygridcolor=:transparent,
      xminorticksvisible=false,
      yminorticksvisible=false,
      xticksvisible=false,
      yticksvisible=false,
      xautolimitmargin=(0.0, 0.0),
      yautolimitmargin=(0.0, 0.0),
      titlefont=:weird,
    ),
  )
  with_theme(empty_theme) do
    fig = Figure()
    ax = Axis(fig[1, 1])
    poly!.(GeoInterface.convert.(Ref(CairoMakie.GeometryBasics), df[:, :geometry]); strokecolor=:black, color=:black)
    ax.title = city_name
    fig
  end
end;

In [ ]:
plot_city(gdf, "Toronto")

In [ ]:
plot_city(gdf, "Montréal")

In [ ]:
plot_city(gdf, "Vancouver")

In [ ]:
plot_city(gdf, "Ottawa")

In [ ]:
plot_city(gdf, "Calgary")

In [ ]:
plot_city(gdf, "Edmonton")

In [ ]:
plot_city(gdf, "Winnipeg")